"""
Dependencies: numpy 1.17, pandas 0.25

userUserNetwork = W; Weighted adjacency matrix, probability user u respond to user v
treads = R
posts = P : user u, creation t, text x
documents = N
questionTendency = average; number of questions by total posts by user u in thread r for topic k
seeking (question) = S; QuestionTendency * log of 1+posts*length
disseminating (answer) = D; 1-Seeking
dictionary = X
topics = K
postTopics (theta) = [0,1]^N*K
topicWords = [0,1]^K*X
SIDR = phi; proportion of seeking by u on topic k by probability for user v responds to user u on topic k
DISR = psi; proportion of disseminating by u on topic k by probability for user u responds to user v on topic k
Benefit = B; utility obtained by user u for topic k; seeking*log of prob v to u on topic k
alpha = marginal benefit of teaching
smoothing = sigma
c_S, c_D = tightness parameters
step = lambda
t = threshold; error

Compute User-User Network
1-Smooth to ensure user responds to each post at most once
QuestionTendency = proportion of questions per topic per thread per weighted-average Q for u

Seeking and Dissemination
1:Extract forum topics: remove stopwords, urls, stem, lemmatize 
2:Infer if post is question or answer: first post and; other post or; has question mark or 5W1H or 1G
3:Compute S and D

Projected Gradient Descent
C_s = participation rate for seeking
C_d = participation rate for dissemination
alpha = learning step
"""

In [ ]:
thread = pd.read_json(r'/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/thread250-0',orient='records',typ='series',convert_dates=False)

In [ ]:
thread = {}
with open ('/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/thread25-0', 'r') as f:
    thread = json.loads(f.read())

In [ ]:
json.load(open('/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/thread25-0','r'))

In [1]:
import pandas as pd
import numpy as np
#import proxmin as px
import json, os

In [265]:
#Import Coursera course discussion forum files, concatenate threads
directory = r'/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/assetpricing-001/'
threads = []
for f in os.listdir(directory):
    thread = json.load(open(os.path.join(directory,f),'r'))
    posts = pd.DataFrame(thread['posts'])
    posts['parent_id'] = posts.sort_values(['thread_id','order'])['user_id'].shift(+1)
    posts['id'].apply(pd.to_numeric)
    comments = pd.DataFrame(thread['comments'])
    threads.append(posts)
    posts.set_index('id',inplace=True)
    if comments.size > 0:
        comments['post_id'].apply(pd.to_numeric)
        comments.set_index('id',inplace=True)
        comments['parent_id'] = pd.Series(comments['post_id'].apply(lambda x: posts.loc[x, 'user_id']))
        threads.append(comments)
    else:
        continue
df = pd.concat(threads)

In [ ]:
ids = []
for thread in threads:
    if 'id' in thread.columns:
        ids.append(thread['id'])
group = list(pd.concat(ids))
assert(len(group) - len(list(set(group))) == 0)

In [216]:
df.set_index('id',inplace=True)

In [270]:
df[df['comment_text'].isna() == False].sort_values(['thread_id','order'])

,order,thread_id,user_id,post_time,edit_time,deleted,is_spam,original,stickied,approved,...,_viewer_can_vote,_viewer_can_edit,_user_title,_user_full_name,_user_profile_link,link,_reporter_link,parent_id,comment_text,post_id
id,,,,,,,,,,,,,,,,,,,,,
25,13.0,1.0,2387060.0,1.379801e+09,-1.0,1.0,0.0,0.0,0.0,0.0,...,True,False,Student,Anonymous,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,4624949.0,,NaN
40,15.0,1.0,4726043.0,1.379816e+09,-1.0,1.0,0.0,0.0,0.0,0.0,...,True,False,Student,Anonymous,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,3067994.0,,NaN
1,NaN,1.0,3823499.0,1.379784e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Staff,N. Aaron Pancost,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,350731.0,Prior coursework in stochastic calculus is not...,4.0
2,NaN,1.0,3823499.0,1.379787e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Staff,N. Aaron Pancost,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,0.0,A good textbook for learning stochastic calcul...,11.0
4,NaN,1.0,0.0,1.379809e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,NaN,NaN,NaN,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,0.0,"Steven's ""Stochastic Calculus for Finance II"" ...",11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,NaN,681.0,4504910.0,1.388487e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Israel Arroyo,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,1421891.0,"<p>Lovely, merry Christmas to you too, Nina!</...",2325.0
748,NaN,681.0,3065781.0,1.388608e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Alexandre Filipe Pereira Neves,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,1421891.0,That is the sweetest geek thing that I have ev...,2325.0
738,NaN,682.0,4894965.0,1.387989e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Stephen Stohs,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,3203020.0,"Many thanks, and I promise to still be listeni...",2315.0


In [243]:
df[df['comment_text'].isnull() == False]['comment_text'].size

858

In [407]:
total_posts = 0
for x in threads:
    total_posts += len(x)
print(total_posts)

3158


In [282]:
df.loc[2325,'user_id']

1421891.0

In [271]:
df[df['post_text'].str.contains('textbook') == True]

,order,thread_id,user_id,post_time,edit_time,deleted,is_spam,original,stickied,approved,...,_viewer_can_vote,_viewer_can_edit,_user_title,_user_full_name,_user_profile_link,link,_reporter_link,parent_id,comment_text,post_id
id,,,,,,,,,,,,,,,,,,,,,
11,9.0,1.0,0.0,1.379787e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,NaN,NaN,NaN,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,2519276.0,NaN,NaN
58,20.0,1.0,2404229.0,1.379858e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Ivan Sutoris,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,4613864.0,NaN,NaN
49,2.0,10.0,125862.0,1.379827e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Gene Beidl,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,0.0,NaN,NaN
700,2.0,193.0,3549836.0,1.381216e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Drew,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,3549836.0,NaN,NaN
1837,7.0,527.0,336852.0,1.384860e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Matthias,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,4062627.0,NaN,NaN
1568,8.0,451.0,4129036.0,1.383610e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Richard Louis Rivero,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,4894965.0,NaN,NaN
1548,1.0,453.0,2898346.0,1.383574e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Student,Howard Lee,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,NaN,NaN,NaN
809,1.0,235.0,4129036.0,1.381477e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Student,Richard Louis Rivero,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,NaN,NaN,NaN
1089,1.0,319.0,0.0,1.382097e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,NaN,NaN,NaN,https://class.coursera.org/assetpricing-001/fo...,https://class.coursera.org/assetpricing-001/he...,NaN,NaN,NaN


In [235]:
df[df.isnull() == False].index

Float64Index([  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
                nan,
              ...
                nan,   nan,   nan,   nan,   nan,   nan, 263.0, 306.0,   nan,
                nan],
             dtype='float64', name='id', length=3158)

In [496]:
import re

def q_a(x):
    qWords = re.compile(r'[\w\W]*(who|what|where|when|why|how|\?)[\w\W]*')
    if re.search(qWords, x):
        return 'question'
    elif x != '':
        return 'answer'
    else:
        return None
        
df['q_a'] = df['post_text'].apply(lambda x: q_a(x) if type(x) == str else np.nan)

In [ ]:
# EdX transforms
#df['_id'] = df['_id'].transform(lambda x: x['$oid'])
#df['parent_id'] = df['parent_id'].transform(lambda x: x['$oid'] if type(x) == dict else np.nan)

In [ ]:
# EdX transform
#df.set_index('_id', inplace=True)
#df['parent_author_id'] = pd.Series(df['parent_id'].apply(lambda x: df.loc[x, 'author_id'] if type(x) == str else 0), dtype='Int64')
#df.reset_index(inplace=True)

In [200]:
df.to_csv('assetpricing-forum.csv')

In [ ]:
#df['post_text'] = df['post_text'].transform(lambda x: x if type(str) else "")

In [246]:
# _type : either CommentThread (i.e., initiation) or Comment (i.e., reply)
InitiationsReplies = pd.crosstab(df['user_id'], df['q_a'])
#InitiationsReplies.head()

In [410]:
# https://github.com/susanli2016/NLP-with-Python/blob/master/LDA_news_headlines.ipynb
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
#nltk.download('wordnet')

In [411]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [412]:
len(df['post_text'])

3158

In [315]:
#print(WordNetLemmatizer().lemmatize(gensim.utils.simple_preprocess(df['post_text'][1])[7], pos='v'))

In [ ]:
#gensim.utils.simple_preprocess(df['post_text'][1])

In [413]:
processed_docs = df['post_text'].map(lambda x: preprocess(x) if type(x) == str else [])

In [414]:
processed_docs[:10]

1968                                         [miss, week]
1985    [schedul, issu, hangout, week, hangout, week, ...
1819    [hello, know, battlehors, cours, matlab, mayb,...
1821                           [ricardo, interest, thank]
1822                                     [interest, post]
1971    [homework, price, drop, nbsp, nbsp, import, de...
1972    [hola, juan, ricardo, thank, post, use, port, ...
1973    [juan, thank, tri, code, honest, matlab, sure,...
66      [span, cours, dash, board, cours, start, sept,...
798       [student, chicago, start, cours, week, earlier]
Name: post_text, dtype: object

In [415]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
#dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [416]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [417]:
len(processed_docs)

3158

In [418]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [419]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"return" + 0.013*"price" + 0.012*"nbsp" + 0.012*"time" + 0.011*"mean" + 0.011*"portfolio" + 0.009*"answer" + 0.007*"question" + 0.007*"thank" + 0.007*"asset"
Topic: 1 
Words: 0.019*"question" + 0.015*"nbsp" + 0.012*"thank" + 0.011*"return" + 0.010*"answer" + 0.009*"sigma" + 0.009*"time" + 0.006*"lambda" + 0.006*"valu" + 0.006*"asset"
Topic: 2 
Words: 0.028*"question" + 0.019*"answer" + 0.014*"return" + 0.013*"quiz" + 0.012*"nbsp" + 0.012*"thank" + 0.009*"cours" + 0.008*"mean" + 0.008*"week" + 0.007*"think"
Topic: 3 
Words: 0.017*"return" + 0.014*"nbsp" + 0.012*"think" + 0.010*"time" + 0.010*"cours" + 0.009*"price" + 0.008*"risk" + 0.008*"expect" + 0.008*"thank" + 0.007*"question"
Topic: 4 
Words: 0.175*"nbsp" + 0.012*"mean" + 0.011*"return" + 0.008*"portfolio" + 0.007*"sigma" + 0.007*"beta" + 0.007*"think" + 0.006*"factor" + 0.006*"price" + 0.006*"market"
Topic: 5 
Words: 0.025*"right" + 0.019*"leav" + 0.013*"http" + 0.010*"span" + 0.007*"nbsp" + 0.007*"correct" 

In [422]:
topicScores = [lda_model.get_document_topics(x) for x in bow_corpus]

In [423]:
topics = pd.DataFrame([max(x, key=lambda y: y[1]) for x in topicScores])

In [424]:
df.insert(2, 'topics', topics[0])
#df.drop('topics', 1)
#topics.shape

In [425]:
len(df.post_text)

3158

In [427]:
df.columns

Index(['order', 'thread_id', 'topics', 'user_id', 'post_time', 'edit_time',
       'deleted', 'is_spam', 'original', 'stickied', 'approved', 'anonymous',
       'votes', 'post_text', 'user_agent', 'text_type', '_viewer_can_vote',
       '_viewer_can_edit', '_user_title', '_user_full_name',
       '_user_profile_link', 'link', '_reporter_link', 'parent_id',
       'comment_text', 'id', 'post_id', 'q_a'],
      dtype='object')

In [428]:
"""Create the adjacency matrix using _id for index and column labels"""
userNetwork = pd.crosstab(df.user_id,df.parent_id)
idx = userNetwork.columns.union(userNetwork.index)
userNetwork = userNetwork.reindex(index=idx, columns=idx, fill_value=0)

In [429]:
userNetwork = pd.DataFrame(userNetwork.sort_index(axis=0).sort_index(axis=1))

In [447]:
userNetwork.drop(0, axis=0, inplace=True)
userNetwork.drop(0, axis=1, inplace=True)
userNetwork.shape

(362, 362)

In [448]:
totalResponses = userNetwork.sum(axis=1)
totalPosts = userNetwork.sum(axis=0).transpose()

In [482]:
w_UserNetwork_full = userNetwork*totalResponses/totalPosts

In [483]:
w_UserNetwork_full.mean(0).head()

1652.0    0.002762
2495.0    0.002762
4920.0    0.002762
5311.0    0.008287
6082.0    0.000000
dtype: float64

In [451]:
userTopics = pd.crosstab(df['user_id'], [df['q_a'], df['topics']])

In [452]:
userTopics.shape

(386, 20)

In [453]:
postTopics = userTopics.sum(0)/userTopics.sum(0).sum()

In [454]:
postTopics

q_a       topics
answer    0         0.162324
          1         0.038018
          2         0.033746
          3         0.024349
          4         0.039727
          5         0.015378
          6         0.035882
          7         0.012815
          8         0.026912
          9         0.038018
question  0         0.219137
          1         0.040581
          2         0.038872
          3         0.037164
          4         0.054677
          5         0.022213
          6         0.052542
          7         0.017941
          8         0.036309
          9         0.053396
dtype: float64

In [455]:
postingTendency = userTopics['question']*postTopics.sum()+userTopics['answer']*postTopics.sum()
questionTendency = (userTopics['question']*postTopics.sum())/postingTendency.sum()

In [456]:
questionTendency.shape

(386, 10)

In [488]:
#userTopics.columns = userTopics.columns.droplevel(0)
disseminating_full = (1-questionTendency * np.log(1+postingTendency))
seeking_full = (questionTendency * np.log(1+postingTendency))

In [468]:
idx = list(set(userNetwork.index.values).intersection(disseminating_full.index.values))

In [489]:
disseminating = disseminating_full.filter(items=idx,axis=0)
seeking = seeking_full.filter(items=idx,axis=0)
f_UserNetwork = w_UserNetwork_full.filter(items=idx,axis=0).filter(items=idx,axis=1)

In [478]:
f_UserNetwork.shape

(351, 351)

In [490]:
threshold = .001
alpha = .4
beta = 0.8
c_S = 1.25
c_D = 0.75
step = 0.1
rho = 1
N = f_UserNetwork.shape[0]
z1 = lambda1 = np.zeros(seeking.T.shape).astype('float64')
z2 = lambda2 = np.zeros(disseminating.shape).astype('float64')
W = W_hat = W_obs = f_UserNetwork.fillna(0).astype('float64').to_numpy()
W_opt = W_prime = np.zeros(W.shape)
D = disseminating.fillna(0).astype('float64').to_numpy()
S = seeking.fillna(0).astype('float64').to_numpy()

def SIDR(s,d,w):
    phi = np.zeros(s.shape)
    for u in range(0,s.shape[0]):
        for k in range(0,s.shape[1]):
            phi[u,k] = s[u,k]/1+(w[:,u].sum()*d[:,k].sum())
    return phi

def DISR(s,d,w):
    psi = np.zeros(d.shape)
    for u in range(d.shape[0]):
        for k in range(d.shape[1]):
            psi[u,k] = d[u,k]/1+(w[u,:].sum()*s[:,k].sum())
    return psi

    
#SIDR.replace(to_replace=0,value=1)
#DISR.replace(to_replace=0,value=1)
P = (S/(1+(c_S*SIDR(S,D,W)))).T
Q = (D/(1+(c_D*DISR(S,D,W))))

In [491]:
np.seterr(all="raise")

def benefit(X):
    b = np.zeros(D.shape)
    for u in range(D.shape[0]):
        for k in range(D.shape[1]):
            b[u,k] = (S[u,k]*np.log(1+X[:,u].sum()*D[:,k].sum()) + alpha*D[u,k]*np.log(1+X[u,:].sum()*S[:,k].sum()))
    return b

def proj(X):
    """Projection step"""
    return np.clip(np.subtract(X,np.diag(np.diag(X))),0,1)
    
def grad(X):
    """Proximal gradient step"""
    return (np.add((rho*np.dot(D,(np.add(np.subtract(np.dot(D.T,X),P),np.subtract(z1,lambda1))))),
            (rho*np.dot((np.subtract(np.subtract(np.dot(X,S),Q),np.add(z2,lambda2))),S.T))))

g = benefit(W).sum().sum()
g_hat = 1
i = 0

while (g - g_hat)/np.abs(g_hat) >= threshold:
    i += 1
    for u in range(W.shape[0]):
        for v in range(W.shape[1]):
            W_prime[u,v] = (
                (D[u].sum()*S[v].sum()/(1+(W[:,v].sum()*D.sum().sum()))
                +alpha*D[u].sum()*S[v].sum()/(1+(W[u,:].sum()*S.sum().sum())))
                /N
            )
    W_hat = (W + step * W_prime)
    W = proj(grad(W_hat))
    g_hat = g
    g = benefit(W).sum().sum()
    z1 = np.clip(np.add(np.add((-1*np.dot(D.T,W)),P),lambda1),None,0)
    z2 = np.clip(np.add(np.subtract(np.dot(W,S),Q),lambda2),None,0)
    lambda1 -= np.subtract(np.add(np.dot(D.T,W),P),z1)
    lambda2 += np.subtract(np.subtract(np.dot(W,S),Q),z2)
    step *= beta

print("Iterations: "+str(i))
print("Observed learning benefit: "+str(benefit(W_obs).sum().sum()/N))
print("Optimized learning benefit: "+str(g_hat/N))

Iterations: 3
Observed learning benefit: 3.717891930859539
Optimized learning benefit: 21.520443400507478


In [301]:
from numpy import linalg as la
import networkx as nx

In [404]:
nx.local_efficiency(nx.from_numpy_array(W))

0.9998404152418651

In [403]:
nx.global_efficiency(nx.from_numpy_array(W))

0.9998377007294192

In [304]:
np.max(la.eig(W_obs)[0]).real

79.09564507997825

In [305]:
np.max(la.eig(W)[0]).real

330.0000000000004

In [492]:
np.savetxt('assetpricing-optimized.csv',W,delimiter=',')